In [1]:
import pandas as pd
import random 
import copy

from meta_kg.utils.py_io import *

In [36]:
clutrr = read_jsonl("data/clutrr/dev.jsonl")

clutrr_4 = [x for x in clutrr if len(x["facts"]) == 4]
clutrr_6 = [x for x in clutrr if len(x["facts"]) == 6]

In [37]:
rels = [
    "son", "daughter",
    "brother", "sister",
    "father", "mother",
    "husband", "wife",
    "grandfather", "grandmother",
    "grandson", "granddaughter",
    "uncle", "aunt",
    "son-in-law", "daughter-in-law",
    "father-in-law", "mother-in-law",
    "brother-in-law", "sister-in-law",
    "nephew", "niece"
]

persons = [
    'A', 'B', 'C', 'D', 
    'H', 'J', 'K', 'L', 
    'M', 'N', 'O', 'P', 
    'Q', 'R', 'S', 'T',
    'V', 'X', 'Y', 'Z',]

def get_knowledge(tokens):
    entitiy = []
    relation = None
    for tok in tokens:
        if tok.isdigit():
            entitiy.append(persons[int(tok)-1])
        if tok in rels:
            relation = tok
    assert len(entitiy) == 2
    if relation is None:
        print(tokens)
    return ' '.join(entitiy), relation


In [38]:
def simplify(dataset):
    simple_dataset = copy.deepcopy(dataset)
    for data in simple_dataset:
        facts = []
        for fact in data['facts']:
            tokens = fact.split()
            entity, relation = get_knowledge(tokens)
            facts.append((entity, relation))
        data['facts'] = facts
        for question in data['questions']:
            answer = question[1]
            tokens = answer.split()
            entity, relation = get_knowledge(tokens)
            question[0] = f"How are {entity[0]} and {entity[-1]} related to each other ?"
            question[1] = entity
            assert len(question[0].split()) == 10
            question.append(relation)
    return simple_dataset

In [39]:
simple_clutrr_4 = simplify(clutrr_4)

In [40]:
write_jsonl(simple_clutrr_4, "data/clutrr_4_hop/dev.jsonl")

In [41]:
simple_clutrr_6 = simplify(clutrr_6)

In [42]:
write_jsonl(simple_clutrr_6, "data/clutrr_6_hop/dev.jsonl")